In [1]:
from dateutil.relativedelta import *
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *
from sklearn.preprocessing import LabelEncoder

import conf
from UI.LOG import *
import cv2
from aml.train_pipeline import *  
from aml.train_pipeline import *
import pandas as pd

import os
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor,FasterRCNN_ResNet50_FPN_Weights
from torchinfo import summary

# from torchvision.models.detection import ssd300_vgg16,SSD300_VGG16_Weights
from torchvision.models.detection.ssd import SSDClassificationHead,SSD300_VGG16_Weights,det_utils
from torchvision.models.detection import ssd300_vgg16

import torch

import aml.model_using as model_using
import aml.support_func as support_funcS
import aml.time_mesuarment as time_mesuarment
import sys

import aml.managers as managers
import aml.img_processing as img_processing
import random
import numpy as np
import pprint
from torchinfo import summary
from aml.img_processing import *

from PIL import Image
import aml.models as models
import matplotlib.pyplot as plt
from pprint import pprint as Print
from PIL import Image
import warnings
from torchvision.utils import draw_bounding_boxes  
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image
from torchvision.ops import nms 
from torchmetrics.detection.mean_ap import MeanAveragePrecision as mAP
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
from IPython.display import IFrame, display, HTML
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from data_manip import label_encode_feautures_
import catboost
from io import StringIO 
import sys
from sklearn.metrics import roc_auc_score
from catboost import utils
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
from tqdm import tqdm
import lightgbm as lgb
tqdm.pandas()

def simple_plot(x,y,title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(x,y)
    ax.set_title(title)
    return fig,ax
def Gini(fpr,tpr):
    return 2*metrics.auc(fpr,tpr)-1
def plot_gxb_train_results(results):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(results['train-auc-mean'],label= 'train-auc-mean')
    ax.plot(results['test-auc-mean'],label= 'test-auc-mean')
    ax.legend()
    return fig,ax


In [2]:
X = pd.read_csv(conf.X_train_dataset,index_col=False)
Y = pd.read_csv(conf.Y_train_dataset,index_col=False)

In [3]:
print(X.shape)

(1199861, 565)


In [4]:
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, train_size=0.8, random_state=1234)
features_names = [el for el in X]
cat_features = np.setdiff1d(label_encode_feautures_,np.setdiff1d(label_encode_feautures_,features_names))
print(cat_features)

['zip_code']


In [5]:
params = {
    'learning_rate': 0.01,
    'max_depth': -1,
    'boosting_type': 'gbdt',
    'n_estimators': 4000,
    'scale_pos_weight': np.sum(np.where(Y==0))/np.sum(np.where(Y==1)),
    'objective':'binary',
    'n_jobs':8,
    'seed':0,
    'subsample': 0.9,
}    
model = lgb.LGBMClassifier(**params)

In [6]:
model.fit(X, Y)

/home/user/python3venvs/ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/user/python3venvs/ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[LightGBM] [Info] Number of positive: 231608, number of negative: 968253
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099861 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4042
[LightGBM] [Info] Number of data points in the train set: 1199861, number of used features: 506
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.193029 -> initscore=-1.430447
[LightGBM] [Info] Start training from score -1.430447


LGBMClassifier(learning_rate=0.01, n_estimators=4000, n_jobs=8,
               objective='binary', scale_pos_weight=4.172730101649272, seed=0,
               subsample=0.9)

In [7]:
preds_ = model.predict(X_validation)
probas_ = model.predict_proba(X_validation)
fpr,tpr,tr = metrics.roc_curve(y_validation,probas_[:,1],pos_label=1)
print('AUC {}'.format(metrics.auc(fpr,tpr)))
print('Gini {}'.format(Gini(fpr,tpr)))

AUC 0.7468432725874995
Gini 0.493686545174999


In [9]:
X_eval = pd.read_csv(conf.X_test_dataset,index_col=False)
print(X_eval.shape)

(590977, 563)


In [11]:
ns1 = [el for el in X]
ns2 = [el for el in X_eval]
names_diff1 = np.setdiff1d(ns1,ns2)
names_diff2=  np.setdiff1d(ns2,ns1)
print(names_diff2)
# X_eval.drop(columns=names_diff,inplace=True)

['emp_length_and_home_ownership_38' 'emp_length_and_home_ownership_50'
 'purpose_and_home_ownership_33' 'purpose_and_home_ownership_51'
 'purpose_and_home_ownership_81']


KeyError: "['emp_length_and_home_ownership_38', 'emp_length_and_home_ownership_50', 'purpose_and_home_ownership_33', 'purpose_and_home_ownership_51', 'purpose_and_home_ownership_81'] not found in axis"

: 

In [8]:
def make_submission(positive_probabilities):
    d = {'index': [i for i in range(0,len(positive_probabilities))], 'loan_status': positive_probabilities}
    o_df = pd.DataFrame(data=d)
    submission_csv = os.path.join('answer.csv')
    submission_zip = os.path.join(conf.data_folder,'answer.zip')
    o_df.to_csv(submission_csv,index=False)
    os.system('rm {}'.format(submission_zip))
    os.system('zip {} {}'.format(submission_zip, submission_csv))
eval_preds_ = model.predict(X_eval)
eval_proba_ = model.predict_proba(X_eval)
probability_of_positive_class = eval_proba_[:,1]
probability_of_negative_class = eval_proba_[:,0]
make_submission(positive_probabilities=probability_of_positive_class)

ValueError: Number of features of the model must match the input. Model n_features_ is 565 and input n_features is 563